In [1]:
# comparison of various conv1d models on the SPEECHCOMMANDS dataset
# classic / fgn trained from scratch / converted

In [2]:
%load_ext autoreload
%autoreload 2
# Disable jedi autocompleter
%config Completer.use_jedi = False

In [3]:
import matplotlib as mpl
# set this 'backend' when using jupyter; do this before importing pyplot
mpl.use('nbagg')
import matplotlib.pyplot as plt
mpl.rcParams['figure.figsize'] = (8.0, 6.0)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
import sys
sys.path.append('/home/felix/Research/Adversarial Research/FGN---Research/')
import Finite_Gaussian_Network_lib as fgnl
import Finite_Gaussian_Network_lib.fgn_helper_lib as fgnh

In [7]:
# load dataset
(train_loader, val_loader, test_loader) = fgnh.SpeechCommands_Dataloaders(resample_rate = 8000,
                                                                          batch_size = 32,
                                                                          batchsize_for_val =32,
                                                                          num_workers=5, 
                                                                          pin_memory=True)

In [37]:
# functions to test models
def number_of_correct(pred, target):
    # count number of correct predictions
    return pred.squeeze().eq(target).sum().item()


def get_likely_index(tensor):
    # find most likely label index for each element in the batch
    return tensor.argmax(dim=-1)


def test(model, loader):
    model.eval()
    correct = 0
    for data, target in loader:

        data = data.to(device)
        target = target.to(device)

        # apply transform and model on whole batch directly on device
        output = model(data)

        pred = get_likely_index(output)
        correct += number_of_correct(pred, target)

    print(f"Accuracy: {correct}/{len(loader.dataset)} ({100. * correct / len(loader.dataset):.0f}%)\n")

In [9]:
# define model classes

## classic model
class M5(nn.Module):
    def __init__(self, n_input=1, n_output=35, stride=16, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=2)
    
# FGN model    
class FGN_M5(nn.Module):
    
    # changes:
    # nn.Conv1d -> fgnl.FGN_Conv1d
    # added g to conv inputs and outputs
    # make sure you pass g through the same pooling steps as x
    
    def __init__(self, n_input=1, n_output=35, stride=16, n_channel=32):
        super().__init__()
        self.fgn_conv1 = fgnl.FGN_Conv1d(in_channels=n_input, out_channels=n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.fgn_conv2 = fgnl.FGN_Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.fgn_conv3 = fgnl.FGN_Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.fgn_conv4 = fgnl.FGN_Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)
        
        #TODO change to self.pool1d_fgn() for each pooling of Gs

    def forward(self, x):
        x, g = self.fgn_conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        g = self.pool1(g)
        x, g = self.fgn_conv2(x, g)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        g = self.pool2(g)
        x, g = self.fgn_conv3(x ,g)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        g = self.pool3(g)
        x, _ = self.fgn_conv4(x, g)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=2)    

In [10]:
# load models
save_path = '../Experiments/sample_SPEECHCOMMANDS_models/'

classic_model_name= 'sample_classic_model_SPEECHCOMMANDS'
fgn_model_name = 'sample_FGN_model_SPEECHCOMMANDS'

In [11]:
classic_model = M5()
classic_model.load_state_dict(torch.load(save_path+classic_model_name+'_state_dict.pth'))
classic_model.to(device)

/home/felix/.local/lib/python3.6/site-packages/torch/cuda/__init__.py:125: UserWarning: 
GeForce RTX 2080 Ti with CUDA capability sm_75 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the GeForce RTX 2080 Ti GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


M5(
  (conv1): Conv1d(1, 32, kernel_size=(80,), stride=(16,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=64, out_features=35, bias=True)
)

In [12]:
fgn_model = FGN_M5()
fgn_model.load_state_dict(torch.load(save_path+fgn_model_name+'_state_dict.pth'))
fgn_model.to(device)

FGN_M5(
  (fgn_conv1): FGN_Conv1d(
    (Conv1d): Conv1d(1, 32, kernel_size=(80,), stride=(16,))
  )
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fgn_conv2): FGN_Conv1d(
    (Conv1d): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  )
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fgn_conv3): FGN_Conv1d(
    (Conv1d): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  )
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fgn_conv4): FGN_Conv1d(
    (Conv1d): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  )
  (bn4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): Ma

In [13]:
# test models
test(classic_model, train_loader)
test(classic_model, val_loader)
test(classic_model, test_loader)

Accuracy: 76655/84843 (90%)

Accuracy: 8755/9981 (88%)

Accuracy: 9468/11005 (86%)



In [14]:
test(fgn_model, train_loader)
test(fgn_model, val_loader)
test(fgn_model, test_loader)

Accuracy: 73489/84843 (87%)

Accuracy: 8572/9981 (86%)

Accuracy: 9244/11005 (84%)



In [15]:
import copy

In [16]:
converted_model = copy.deepcopy(fgn_model)

In [17]:
test(converted_model, train_loader)
test(converted_model, val_loader)
test(converted_model, test_loader)

Accuracy: 73489/84843 (87%)

Accuracy: 8572/9981 (86%)

Accuracy: 9244/11005 (84%)



In [18]:
for classic_model_layer,fgn_model_layer in zip(classic_model.children(), converted_model.children()):

    if type(fgn_model_layer)==fgnl.FGN_Conv1d:
        print('converting conv layer')
        fgnl.convert_layer_conv1D_to_fgn(classic_model_layer,fgn_model_layer)
    else:
        print('transfering state_dicts')
        fgn_model_layer.load_state_dict(classic_model_layer.state_dict())
        

converting conv layer
transfering state_dicts
transfering state_dicts
converting conv layer
transfering state_dicts
transfering state_dicts
converting conv layer
transfering state_dicts
transfering state_dicts
converting conv layer
transfering state_dicts
transfering state_dicts
transfering state_dicts


In [39]:
# check that the converted model matches classic behavior
test(converted_model, train_loader)
test(converted_model, val_loader)
test(converted_model, test_loader)

Accuracy: 76655/84843 (90%)

Accuracy: 8755/9981 (88%)

Accuracy: 9468/11005 (86%)



In [20]:
# check that the original model hasn't changed
test(converted_model, train_loader)
test(converted_model, val_loader)
test(converted_model, test_loader)

Accuracy: 76655/84843 (90%)

Accuracy: 8755/9981 (88%)

Accuracy: 9468/11005 (86%)



In [21]:
# retrain a bit

In [32]:
import torch.optim as optim

In [30]:
import IPython.display as ipd
from tqdm.notebook import tqdm

In [22]:
def train(model, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        data = data.to(device)
        target = target.to(device)

        # apply transform and model on whole batch directly on device
        output = model(data)

        # negative log-likelihood for a tensor of size (batch x 1 x n_output)
        loss = F.nll_loss(output.squeeze(), target) + lmbda_sigma*fgnl.sigmas_loss(model, covar_type='sphere')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print training stats
        if batch_idx % log_interval == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

        # update progress bar
        pbar.update(pbar_update)
        # record loss
        losses.append(loss.item())

In [52]:
fast_retrained_converted_model = copy.deepcopy(converted_model)

In [53]:
lmbda_sigma = 1e-5
optimizer = optim.Adam(fast_retrained_converted_model.parameters(), lr=0.001, weight_decay=0.0001)
# note LR is 10x smaller because the models have already beenm trained for 20 epochs

In [54]:
# quick retrain the converted model
log_interval = 20
n_epoch = 1

pbar_update = 1 / (len(train_loader) + len(test_loader))
losses = []

# The transform needs to live on the same device as the model and the data.
# transform = transform.to(device)
with tqdm(total=n_epoch) as pbar:
    for epoch in range(1, n_epoch + 1):
        train(fast_retrained_converted_model, epoch, log_interval)
        test(fast_retrained_converted_model, test_loader)
        scheduler.step()

Train Epoch: 1 [0/84843 (0%)]	Loss: 569473.375000
Train Epoch: 1 [640/84843 (1%)]	Loss: 0.607516
Train Epoch: 1 [1280/84843 (2%)]	Loss: 0.570802
Train Epoch: 1 [1920/84843 (2%)]	Loss: 0.648478
Train Epoch: 1 [2560/84843 (3%)]	Loss: 0.708704
Train Epoch: 1 [3200/84843 (4%)]	Loss: 0.730980
Train Epoch: 1 [3840/84843 (5%)]	Loss: 0.626619
Train Epoch: 1 [4480/84843 (5%)]	Loss: 0.695850
Train Epoch: 1 [5120/84843 (6%)]	Loss: 0.983909
Train Epoch: 1 [5760/84843 (7%)]	Loss: 0.553858
Train Epoch: 1 [6400/84843 (8%)]	Loss: 0.457170
Train Epoch: 1 [7040/84843 (8%)]	Loss: 1.158071
Train Epoch: 1 [7680/84843 (9%)]	Loss: 0.466663
Train Epoch: 1 [8320/84843 (10%)]	Loss: 0.771917
Train Epoch: 1 [8960/84843 (11%)]	Loss: 0.537751
Train Epoch: 1 [9600/84843 (11%)]	Loss: 0.492841
Train Epoch: 1 [10240/84843 (12%)]	Loss: 0.779520
Train Epoch: 1 [10880/84843 (13%)]	Loss: 0.670316
Train Epoch: 1 [11520/84843 (14%)]	Loss: 0.635499
Train Epoch: 1 [12160/84843 (14%)]	Loss: 0.592665
Train Epoch: 1 [12800/84843 

In [55]:
# check that the fast retrained converted model improved and is good.
test(fast_retrained_converted_model, train_loader)
test(fast_retrained_converted_model, val_loader)
test(fast_retrained_converted_model, test_loader)

Accuracy: 76173/84843 (90%)

Accuracy: 8702/9981 (87%)

Accuracy: 9418/11005 (86%)



In [ ]:
# retrain longer

In [56]:
long_retrained_converted_model = copy.deepcopy(fast_retrained_converted_model)

In [57]:
optimizer = optim.Adam(long_retrained_converted_model.parameters(), lr=0.001, weight_decay=0.0001)

In [ ]:
# longer retrain the converted model
log_interval = 20
n_epoch = 20

pbar_update = 1 / (len(train_loader) + len(test_loader))
losses = []

# The transform needs to live on the same device as the model and the data.
# transform = transform.to(device)
with tqdm(total=n_epoch) as pbar:
    for epoch in range(1, n_epoch + 1):
        train(long_retrained_converted_model, epoch, log_interval)
        test(long_retrained_converted_model, test_loader)
        scheduler.step()

Train Epoch: 1 [0/84843 (0%)]	Loss: 0.593730
Train Epoch: 1 [640/84843 (1%)]	Loss: 0.647424
Train Epoch: 1 [1280/84843 (2%)]	Loss: 0.452424
Train Epoch: 1 [1920/84843 (2%)]	Loss: 0.463932
Train Epoch: 1 [2560/84843 (3%)]	Loss: 0.413830
Train Epoch: 1 [3200/84843 (4%)]	Loss: 0.455938
Train Epoch: 1 [3840/84843 (5%)]	Loss: 0.344253
Train Epoch: 1 [4480/84843 (5%)]	Loss: 0.723838
Train Epoch: 1 [5120/84843 (6%)]	Loss: 0.579033
Train Epoch: 1 [5760/84843 (7%)]	Loss: 0.629117
Train Epoch: 1 [6400/84843 (8%)]	Loss: 0.553000
Train Epoch: 1 [7040/84843 (8%)]	Loss: 0.416191
Train Epoch: 1 [7680/84843 (9%)]	Loss: 0.406114
Train Epoch: 1 [8320/84843 (10%)]	Loss: 0.372227
Train Epoch: 1 [8960/84843 (11%)]	Loss: 0.187963
Train Epoch: 1 [9600/84843 (11%)]	Loss: 0.497532
Train Epoch: 1 [10240/84843 (12%)]	Loss: 0.212778
Train Epoch: 1 [10880/84843 (13%)]	Loss: 0.368652
Train Epoch: 1 [11520/84843 (14%)]	Loss: 0.151380
Train Epoch: 1 [12160/84843 (14%)]	Loss: 0.392837
Train Epoch: 1 [12800/84843 (15%)

Train Epoch: 2 [20480/84843 (24%)]	Loss: 0.424053
Train Epoch: 2 [21120/84843 (25%)]	Loss: 0.564802
Train Epoch: 2 [21760/84843 (26%)]	Loss: 0.434176
Train Epoch: 2 [22400/84843 (26%)]	Loss: 0.344183
Train Epoch: 2 [23040/84843 (27%)]	Loss: 0.395888
Train Epoch: 2 [23680/84843 (28%)]	Loss: 0.544507
Train Epoch: 2 [24320/84843 (29%)]	Loss: 0.233617
Train Epoch: 2 [24960/84843 (29%)]	Loss: 0.539178
Train Epoch: 2 [25600/84843 (30%)]	Loss: 0.465553
Train Epoch: 2 [26240/84843 (31%)]	Loss: 0.552109
Train Epoch: 2 [26880/84843 (32%)]	Loss: 0.367343
Train Epoch: 2 [27520/84843 (32%)]	Loss: 0.426576
Train Epoch: 2 [28160/84843 (33%)]	Loss: 0.205420
Train Epoch: 2 [28800/84843 (34%)]	Loss: 0.229924
Train Epoch: 2 [29440/84843 (35%)]	Loss: 0.339696
Train Epoch: 2 [30080/84843 (35%)]	Loss: 0.322260
Train Epoch: 2 [30720/84843 (36%)]	Loss: 0.509919
Train Epoch: 2 [31360/84843 (37%)]	Loss: 0.667047
Train Epoch: 2 [32000/84843 (38%)]	Loss: 0.939134
Train Epoch: 2 [32640/84843 (38%)]	Loss: 0.360841


Train Epoch: 3 [40320/84843 (48%)]	Loss: 0.254444
Train Epoch: 3 [40960/84843 (48%)]	Loss: 0.636647
Train Epoch: 3 [41600/84843 (49%)]	Loss: 0.451768
Train Epoch: 3 [42240/84843 (50%)]	Loss: 0.417810
Train Epoch: 3 [42880/84843 (51%)]	Loss: 0.241567
Train Epoch: 3 [43520/84843 (51%)]	Loss: 0.483856
Train Epoch: 3 [44160/84843 (52%)]	Loss: 0.646982
Train Epoch: 3 [44800/84843 (53%)]	Loss: 0.770773
Train Epoch: 3 [45440/84843 (54%)]	Loss: 0.274732
Train Epoch: 3 [46080/84843 (54%)]	Loss: 0.186158
Train Epoch: 3 [46720/84843 (55%)]	Loss: 0.307415
Train Epoch: 3 [47360/84843 (56%)]	Loss: 0.372190
Train Epoch: 3 [48000/84843 (57%)]	Loss: 0.168942
Train Epoch: 3 [48640/84843 (57%)]	Loss: 0.301063
Train Epoch: 3 [49280/84843 (58%)]	Loss: 0.263631
Train Epoch: 3 [49920/84843 (59%)]	Loss: 0.515767
Train Epoch: 3 [50560/84843 (60%)]	Loss: 0.468115
Train Epoch: 3 [51200/84843 (60%)]	Loss: 0.102583
Train Epoch: 3 [51840/84843 (61%)]	Loss: 0.621563
Train Epoch: 3 [52480/84843 (62%)]	Loss: 0.493441


Train Epoch: 4 [60160/84843 (71%)]	Loss: 0.501250
Train Epoch: 4 [60800/84843 (72%)]	Loss: 0.224827
Train Epoch: 4 [61440/84843 (72%)]	Loss: 0.527216
Train Epoch: 4 [62080/84843 (73%)]	Loss: 0.428892
Train Epoch: 4 [62720/84843 (74%)]	Loss: 0.673670
Train Epoch: 4 [63360/84843 (75%)]	Loss: 0.287131
Train Epoch: 4 [64000/84843 (75%)]	Loss: 0.332425
Train Epoch: 4 [64640/84843 (76%)]	Loss: 0.450771
Train Epoch: 4 [65280/84843 (77%)]	Loss: 0.245984
Train Epoch: 4 [65920/84843 (78%)]	Loss: 0.567181
Train Epoch: 4 [66560/84843 (78%)]	Loss: 0.130967
Train Epoch: 4 [67200/84843 (79%)]	Loss: 0.375433
Train Epoch: 4 [67840/84843 (80%)]	Loss: 0.581069
Train Epoch: 4 [68480/84843 (81%)]	Loss: 0.439048
Train Epoch: 4 [69120/84843 (81%)]	Loss: 0.463379
Train Epoch: 4 [69760/84843 (82%)]	Loss: 0.358943
Train Epoch: 4 [70400/84843 (83%)]	Loss: 0.399621
Train Epoch: 4 [71040/84843 (84%)]	Loss: 0.341854
Train Epoch: 4 [71680/84843 (84%)]	Loss: 0.204438
Train Epoch: 4 [72320/84843 (85%)]	Loss: 0.295025


Train Epoch: 5 [80000/84843 (94%)]	Loss: 0.164629
Train Epoch: 5 [80640/84843 (95%)]	Loss: 0.201807
Train Epoch: 5 [81280/84843 (96%)]	Loss: 0.414738
Train Epoch: 5 [81920/84843 (97%)]	Loss: 0.244289
Train Epoch: 5 [82560/84843 (97%)]	Loss: 0.274602
Train Epoch: 5 [83200/84843 (98%)]	Loss: 0.374336
Train Epoch: 5 [83840/84843 (99%)]	Loss: 0.479721
Train Epoch: 5 [84480/84843 (100%)]	Loss: 0.251939
Accuracy: 9436/11005 (86%)

Train Epoch: 6 [0/84843 (0%)]	Loss: 0.251898
Train Epoch: 6 [640/84843 (1%)]	Loss: 0.213981
Train Epoch: 6 [1280/84843 (2%)]	Loss: 0.325060
Train Epoch: 6 [1920/84843 (2%)]	Loss: 0.219026
Train Epoch: 6 [2560/84843 (3%)]	Loss: 0.613633
Train Epoch: 6 [3200/84843 (4%)]	Loss: 0.296381
Train Epoch: 6 [3840/84843 (5%)]	Loss: 0.239596
Train Epoch: 6 [4480/84843 (5%)]	Loss: 0.459909
Train Epoch: 6 [5120/84843 (6%)]	Loss: 0.517578
Train Epoch: 6 [5760/84843 (7%)]	Loss: 0.256359
Train Epoch: 6 [6400/84843 (8%)]	Loss: 0.504849
Train Epoch: 6 [7040/84843 (8%)]	Loss: 0.327583

Train Epoch: 7 [15360/84843 (18%)]	Loss: 0.198369
Train Epoch: 7 [16000/84843 (19%)]	Loss: 0.342084
Train Epoch: 7 [16640/84843 (20%)]	Loss: 0.363473
Train Epoch: 7 [17280/84843 (20%)]	Loss: 0.119368
Train Epoch: 7 [17920/84843 (21%)]	Loss: 0.168545
Train Epoch: 7 [18560/84843 (22%)]	Loss: 0.472648
Train Epoch: 7 [19200/84843 (23%)]	Loss: 0.489822
Train Epoch: 7 [19840/84843 (23%)]	Loss: 0.113341
Train Epoch: 7 [20480/84843 (24%)]	Loss: 0.363324
Train Epoch: 7 [21120/84843 (25%)]	Loss: 0.573969
Train Epoch: 7 [21760/84843 (26%)]	Loss: 0.456604
Train Epoch: 7 [22400/84843 (26%)]	Loss: 0.537859
Train Epoch: 7 [23040/84843 (27%)]	Loss: 0.439044
Train Epoch: 7 [23680/84843 (28%)]	Loss: 0.355658
Train Epoch: 7 [24320/84843 (29%)]	Loss: 0.458340
Train Epoch: 7 [24960/84843 (29%)]	Loss: 0.103405
Train Epoch: 7 [25600/84843 (30%)]	Loss: 0.485997
Train Epoch: 7 [26240/84843 (31%)]	Loss: 0.256063
Train Epoch: 7 [26880/84843 (32%)]	Loss: 0.354402
Train Epoch: 7 [27520/84843 (32%)]	Loss: 0.235696


Train Epoch: 8 [35200/84843 (41%)]	Loss: 0.524001
Train Epoch: 8 [35840/84843 (42%)]	Loss: 0.232844
Train Epoch: 8 [36480/84843 (43%)]	Loss: 0.317180
Train Epoch: 8 [37120/84843 (44%)]	Loss: 0.773124
Train Epoch: 8 [37760/84843 (44%)]	Loss: 0.236675
Train Epoch: 8 [38400/84843 (45%)]	Loss: 0.515612
Train Epoch: 8 [39040/84843 (46%)]	Loss: 0.368092
Train Epoch: 8 [39680/84843 (47%)]	Loss: 0.224621
Train Epoch: 8 [40320/84843 (48%)]	Loss: 0.063501
Train Epoch: 8 [40960/84843 (48%)]	Loss: 0.338740
Train Epoch: 8 [41600/84843 (49%)]	Loss: 0.343596
Train Epoch: 8 [42240/84843 (50%)]	Loss: 0.436595
Train Epoch: 8 [42880/84843 (51%)]	Loss: 0.286970
Train Epoch: 8 [43520/84843 (51%)]	Loss: 0.339002
Train Epoch: 8 [44160/84843 (52%)]	Loss: 0.128705
Train Epoch: 8 [44800/84843 (53%)]	Loss: 0.672965
Train Epoch: 8 [45440/84843 (54%)]	Loss: 0.680940
Train Epoch: 8 [46080/84843 (54%)]	Loss: 0.386288
Train Epoch: 8 [46720/84843 (55%)]	Loss: 0.450744
Train Epoch: 8 [47360/84843 (56%)]	Loss: 0.332997


Train Epoch: 9 [55040/84843 (65%)]	Loss: 0.242604
Train Epoch: 9 [55680/84843 (66%)]	Loss: 0.247191
Train Epoch: 9 [56320/84843 (66%)]	Loss: 0.297560
Train Epoch: 9 [56960/84843 (67%)]	Loss: 0.431917
Train Epoch: 9 [57600/84843 (68%)]	Loss: 0.161780
Train Epoch: 9 [58240/84843 (69%)]	Loss: 0.385029
Train Epoch: 9 [58880/84843 (69%)]	Loss: 0.081591
Train Epoch: 9 [59520/84843 (70%)]	Loss: 0.339154
Train Epoch: 9 [60160/84843 (71%)]	Loss: 0.431709
Train Epoch: 9 [60800/84843 (72%)]	Loss: 0.643423
Train Epoch: 9 [61440/84843 (72%)]	Loss: 0.380961
Train Epoch: 9 [62080/84843 (73%)]	Loss: 0.362633
Train Epoch: 9 [62720/84843 (74%)]	Loss: 0.204411
Train Epoch: 9 [63360/84843 (75%)]	Loss: 0.262995
Train Epoch: 9 [64000/84843 (75%)]	Loss: 0.437309
Train Epoch: 9 [64640/84843 (76%)]	Loss: 0.420181
Train Epoch: 9 [65280/84843 (77%)]	Loss: 0.402398
Train Epoch: 9 [65920/84843 (78%)]	Loss: 0.369846
Train Epoch: 9 [66560/84843 (78%)]	Loss: 0.410082
Train Epoch: 9 [67200/84843 (79%)]	Loss: 0.268341


Train Epoch: 10 [73600/84843 (87%)]	Loss: 0.885403
Train Epoch: 10 [74240/84843 (87%)]	Loss: 0.210718
Train Epoch: 10 [74880/84843 (88%)]	Loss: 0.261975
Train Epoch: 10 [75520/84843 (89%)]	Loss: 0.381532
Train Epoch: 10 [76160/84843 (90%)]	Loss: 0.215224
Train Epoch: 10 [76800/84843 (90%)]	Loss: 0.092760
Train Epoch: 10 [77440/84843 (91%)]	Loss: 0.508985
Train Epoch: 10 [78080/84843 (92%)]	Loss: 0.349670
Train Epoch: 10 [78720/84843 (93%)]	Loss: 0.332398
Train Epoch: 10 [79360/84843 (94%)]	Loss: 0.470098
Train Epoch: 10 [80000/84843 (94%)]	Loss: 0.226882
Train Epoch: 10 [80640/84843 (95%)]	Loss: 0.446288
Train Epoch: 10 [81280/84843 (96%)]	Loss: 0.518469
Train Epoch: 10 [81920/84843 (97%)]	Loss: 0.613580
Train Epoch: 10 [82560/84843 (97%)]	Loss: 0.182081
Train Epoch: 10 [83200/84843 (98%)]	Loss: 0.223096
Train Epoch: 10 [83840/84843 (99%)]	Loss: 0.366736
Train Epoch: 10 [84480/84843 (100%)]	Loss: 0.350648
Accuracy: 9444/11005 (86%)

Train Epoch: 11 [0/84843 (0%)]	Loss: 0.336942
Train E

Train Epoch: 12 [6400/84843 (8%)]	Loss: 0.381570
Train Epoch: 12 [7040/84843 (8%)]	Loss: 0.697736
Train Epoch: 12 [7680/84843 (9%)]	Loss: 0.196910
Train Epoch: 12 [8320/84843 (10%)]	Loss: 0.262203
Train Epoch: 12 [8960/84843 (11%)]	Loss: 0.513870
Train Epoch: 12 [9600/84843 (11%)]	Loss: 0.264032


In [ ]:
# check that the fast retrained converted model improved and is good.
test(long_retrained_converted_model, train_loader)
test(long_retrained_converted_model, val_loader)
test(long_retrained_converted_model, test_loader)